In [ ]:
!pip install accelerate  # 세션 다시 시작 필요

In [ ]:
!pip install peft

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED=42

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')
sample = pd.read_csv(f'{data_path}sample_submission.csv')
train_gpt = pd.read_csv(f'{data_path}train_gpt.csv')


train_ft = pd.read_csv(data_path + 'train_80000_rule_based.csv')


In [ ]:
import os, torch, tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
!nvidia-smi

Sun Mar  3 00:51:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# PEFT 모델 로드 및 추론
- https://huggingface.co/docs/trl/main/en/use_model

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import json
import torch

# 모델 식별자
model_ID = "megastudyedu/M-SOLAR-10.7B-v1.3"

In [ ]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
saved_model_path = f'{data_path}models/megast_0302/'

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 25.2 MB/s eta 0:00:00


In [ ]:
model_ID = 'megastudyedu/M-SOLAR-10.7B-v1.3'

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    use_nested_quant=False
)

# AdapterModelForCausalLM 클래스로 모델 정의
base_model = AutoModelForCausalLM.from_pretrained(
    model_ID,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
# 기본 모델 hugging face
original_model = AutoModelForCausalLM.from_pretrained(model_ID)
print(original_model)
print('#'*100)

# 학습 전 peft 모델
print(base_model)
print('#'*100)

# 학습 후 peft 모델
adapter_model_name = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/models/megast_0302/'
model = PeftModel.from_pretrained(base_model, adapter_model_name).to(device)
print(model)
print('#'*100)

# 학습 후 merge된 peft 모델
merge_model = model.merge_and_unload()
print(merge_model)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ID)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

# 추론 실험

In [ ]:
import pandas as pd
test = pd.read_csv(f'{data_path}test.csv')
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [ ]:
def generate_response_no_tuned(user_request, max_len=1000):
    conversation = [ {'role': 'user', 'content': user_request} ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, use_cache=True, max_length=max_len)
    output_text = tokenizer.decode(outputs[0])
    output_text = output_text.split("### Assistant:")[1].strip()
    output_text = output_text.replace('\n','').replace('</s>','')

    return output_text

In [ ]:
generate_response_no_tuned(test['질문'][0],1000)

'방청 페인트는 철제 및 철제 기반 구조물을 부식으로부터 보호하기 위해 설계된 특수 페인트입니다. 방청 페인트의 주요 유형은 다음과 같습니다:\n\n1. 에폭시 방청 페인트: 내부 및 외부 표면에 사용하기에 적합한 고강도 및 내마모성 페인트입니다.\n2. 아크릴 방청 페인트: 내부 및 외부 표면에 사용하기에 적합한 저농도 페인트입니다.\n3. 알킬 방청 페인트: 내부 및 외부 표면에 사용하기에 적합한 저농도 페인트입니다.\n4. 에틸 방청 페인트: 내부 및 외부 표면에 사용하기에 적합한 저농도 페인트입니다.\n5. 아크릴 에틸 방청 페인트: 내부 및 외부 표면에 사용하기에 적합한 저농도 페인트입니다.\n\n원목 사이딩은 건물 외벽에 사용되는 자연 나무 판자입니다. 원목 사이딩의 장점은 다음과 같습니다:\n\n1. 미적 매력: 원목 사이딩은 다양한 종류의 나무로 제작되어 다양한 색상과 질감을 제공합니다.\n2. 지속 가능성: 원목 사이딩은 재생 가능한 자연 자원으로 만들어져 환경에 미치는 영향이 적습니다.\n3. 단열 특성: 원목 사이딩은 단열 특성이 좋아 에너지 절약에 도움이 됩니다.\n\n그러나 원목 사이딩의 단점은 다음과 같습니다:\n\n1. 유지 관리: 원목 사이딩은 곰팡이, 부식 및 기타 손상으로부터 보호하기 위해 주기적으로 도장 및 유지 관리가 필요합니다.\n2. 가격: 원목 사이딩은 다른 사이딩 옵션에 비해 비싸다는 단점이 있습니다.\n3. 내구성: 원목 사이딩은 강한 풍속, 빛, 습기에 노출되면 손상될 수 있습니다.</s>'

In [ ]:
def generate_response(user_request, max_len=1000):
    conversation = [ {'role': 'user', 'content': user_request} ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             use_cache=True,
                             max_length=max_len,
                            num_return_sequences=1,
                            do_sample=True,
                            temperature=0.87,
                            top_k=27,
                            top_p=0.7
                             )
    output_text = tokenizer.decode(outputs[0])
    output_text = output_text.split("### Assistant:")[1].strip()
    output_text = output_text.replace('\n','').replace('</s>','')

    return output_text

In [ ]:
generate_response(test['질문'][0],1024)

'방청 페인트의 종류는 다양하며 다양한 목적과 환경에 맞게 설계되었습니다. 일반적인 방청 페인트 유형에는 다음이 있습니다:1. 산화방지 코팅: 산화방지 코팅은 강철, 알루미늄, 구리와 같은 금속 표면을 부식으로부터 보호하는 데 사용됩니다. 산화방지 코팅은 산소 및 수분과 같은 환경 요인으로부터 금속을 보호하기 위해 금속과 반응하여 자기 방식 방지막을 형성합니다.2. 방청 페인트: 방청 페인트는 금속 표면을 부식으로부터 보호하는 데 사용되지만 산화방지 코팅과 달리 자기 방식 방지막을 형성하지는 않습니다. 대신 방청 페인트는 금속 표면을 수분과 산소로부터 보호하기 위해 기계적 장벽을 제공합니다.3. 방청 갈바닌 코팅: 방청 갈바닌 코팅은 갈바닌 강철 표면을 부식으로부터 보호하는 데 사용됩니다. 이러한 코팅은 강철 표면에 부식 방지 층을 형성하여 수분과 산소가 강철과 반응하여 부식을 일으키는 것을 방지합니다.4. 방청 에폭시 코팅: 방청 에폭시 코팅은 방청 페인트와 유사하지만 더 높은 내구성을 가지고 있습니다. 이러한 코팅은 부식에 대한 방지 기능과 함께 더 높은 기계적 응력을 견딜 수 있습니다.원목 사이딩을 사용하면 몇 가지 단점이 있을 수 있습니다:1. 유지보수: 원목 사이딩은 절단 표면에서 부식이 쉽게 발생할 수 있으므로 주기적인 유지보수가 필요합니다. 이는 벗겨지거나 갈라지는 부분을 수리하고 필요한 경우 재페인트하는 것을 의미합니다.2. 내구성: 원목 사이딩은 염수 분위기, 극한 온도, 태양의 자외선 등 다양한 환경 요인에 취약할 수 있습니다. 이러한 요인은 원목의 얼굴을 변색시키고 부러지고 풀어지게 만들 수 있습니다.3. 비용: 원목 사이딩은 설치 및 유지보수 비용이 비싸다는 점에서 비용이 많이 들 수'

In [ ]:
generate_response(test['질문'][0],200)

'방청 페인트는 부식을 방지하고 장시간 내구성을 유지하기 위해 특별히 설계된 페인트의 일종입니다. 방청 페인트의 주요 종류는 다음과 같습니다:1. 아연 도료: 아연 기반 페인트는 부식 방지 성능이 우수하여 방청 도'

In [ ]:
generate_response(test['질문'][0],2048)

'방청 페인트는 다양한 종류가 있습니다. 일반적인 방청 페인트 종류에는 다음이 있습니다:1. 알칼리 방청 페인트: 알칼리 방청 페인트는 알칼리 환경에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 산성 분위기에서도 효과적인 방청 보호를 제공합니다.2. 아마인 방청 페인트: 아마인 방청 페인트는 수용액 내에서 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 물과 같은 액체 환경에서 금속을 보호하는 데 특히 유용합니다.3. 내산 방청 페인트: 내산 방청 페인트는 산성 환경에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 산성 분위기에서도 효과적인 방청 보호를 제공합니다.4. 내알칼리 방청 페인트: 내알칼리 방청 페인트는 알칼리 환경에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 알칼리 분위기에서도 효과적인 방청 보호를 제공합니다.5. 내부식성 방청 페인트: 내부식성 방청 페인트는 금속을 부식하는 물리적 또는 화학적 과정에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 부식에 노출된 금속을 보호하는 데 특히 유용합니다.6. 내연소성 방청 페인트: 내연소성 방청 페인트는 금속을 연소 또는 산화에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 금속을 연소 또는 산화에 노출된 환경에서 보호하는 데 특히 유용합니다.7. 내화학 방청 페인트: 내화학 방청 페인트는 금속을 화학 물질에 노출된 금속을 보호하기 위해 설계된 방청 페인트의 일종입니다. 이러한 페인트는 금속을 화학 물질에 노출된 환경에서 보호하는 데 특히 유용합니다.원목 사이딩은 자연 목재로 만들어진 사이딩 재료로, 주택 외벽을 보호하고 단열하는 데 사용됩니다. 원목 사이딩은 다음과 같은 단점이 있습니다:1. 유지 관리: 원목 사이딩은 균열이 생기지 않도록 정기적으로 유지 관리가 필요합니다. 이는 수명이 짧고 손상이 쉽게 발생할 수 

In [ ]:
generate_response(test['질문'][0],4000)

'방청 페인트는 습기에 노출되어 있거나 습기가 많은 환경에서 사용하는 것이 일반적인 목재 보호 페인트의 한 종류입니다. 방청 페인트의 일반적인 종류에는 다음이 있습니다:1. 오일 기반 방청 페인트: 이 종류의 방청 페인트는 오일 기반 수지와 방청제가 함유된 것으로, 수분을 방출하고 목재를 보호하는 데 도움이 됩니다. 이 페인트는 풍부한 광택과 긴 지속력으로 인해 외부 목재 표면에 인기가 있습니다.2. 수성 방청 페인트: 수성 방청 페인트는 수성 수지와 방청제를 사용하여 제조됩니다. 이 페인트는 오일 기반 페인트에 비해 건조 시간이 빠르고 쉽게 제거할 수 있어 외부 목재 표면에 인기가 있습니다.3. 하이브리드 방청 페인트: 하이브리드 방청 페인트는 오일 기반 페인트와 수성 페인트의 장점을 결합한 것입니다. 이 페인트는 오일 기반 페인트와 같은 광택과 내구성을 가지고 있지만 수성 페인트처럼 쉽게 제거할 수 있습니다.원목 사이딩을 사용하는 것에는 다음과 같은 단점이 있습니다:1. 유지보수 요구 사항: 원목 사이딩은 자연 방식으로 노출되는 것으로, 곰팡이 및 부식으로 인해 손상을 입을 수 있습니다. 따라서 원목 사이딩을 유지하려면 빈번한 깨끗이 씻기와 방청 페인트 또는 스타인 코팅 적용이 필요합니다.2. 수명 단축: 원목 사이딩은 노출된 목재이기 때문에 부식, 흡습, 흡습에 취약합니다. 이로 인해 원목 사이딩의 수명이 단축될 수 있습니다.3. 재료 비용: 원목 사이딩은 다른 재료에 비해 비용이 높을 수 있습니다. 원목 사이딩을 구하고 설치하는 데 드는 비용 외에도 유지 및 보수에 추가 비용이 발생할 수 있습니다.4. 개인 취향에 따른 제한: 원목 사이딩은 일반적으로 벽면에 자연 상태의 목재를 사용하기 때문에 색상 및 외관 옵션이 제한될 수 있습니다. 원하는 색상을 얻으려면 추가 스타인 또는 페인트 코팅이 필요할 수 있습니다.전반적으로 방청 페인트는 습기 및 부식으로부터 목재를 보호하는 데 도움이 되는 반면, 원목 사이딩은 유지 및 보수가 필요하고 비용이 높을 수 있습

In [ ]:
generate_response(test['질문'][0],2000)

'방청 페인트의 주요 유형은 방청 코팅, 방청 코팅, 방청 코팅 등 3가지가 있습니다. 각 유형의 특징은 다음과 같습니다:1. 방청 코팅: 이 유형의 방청 페인트는 철제 표면에 직접 적용하여 부식을 방지하는 데 사용됩니다. 주로 산화 방지 기능이 있는 아연 기반 페인트로 제조됩니다. 이 페인트는 강철 및 알루미늄과 같은 다양한 철제 재료에 적용할 수 있습니다.2. 방청 코팅: 이 유형의 방청 페인트는 주로 방수 및 방청 기능을 결합한 다목적 코팅입니다. 주로 철제 표면에 직접 적용하여 부식을 방지하고 물이나 습기가 표면에 침투하는 것을 방지합니다. 이 페인트는 강철 및 알루미늄과 같은 다양한 철제 재료에 적용할 수 있습니다.3. 방청 코팅: 이 유형의 방청 페인트는 주로 철제 표면에 직접 적용하여 부식을 방지하는 데 사용되는 방청 코팅의 하위 유형입니다. 주로 철제 표면에 직접 적용하여 부식을 방지하는 데 사용됩니다. 이 페인트는 강철 및 알루미늄과 같은 다양한 철제 재료에 적용할 수 있습니다.원목 사이딩을 사용하면 다양한 장점이 있습니다. 하지만 단점도 있습니다. 원목 사이딩의 단점은 다음과 같습니다:1. 유지 관리: 원목 사이딩은 기온 변화, 햇빛, 비 등 자연 요인으로 인해 변색이나 흐름이 발생할 수 있으므로 정기적인 유지 관리가 필요합니다.2. 부식: 원목 사이딩은 습기와 충격에 노출되면 부식될 수 있습니다. 따라서 특히 강우량이 많은 지역에서는 사이딩 시스템을 유지하고 보수하는 것이 중요합니다.3. 비용: 원목 사이딩은 다른 재료에 비해 비용이 높을 수 있습니다. 또한 유지 관리 및 보수 비용도 고려해야 합니다.4. 재활용 가능성: 원목 사이딩은 재활용이 어려워 일반적으로 쓰레기로 처리됩니다. 이는 환경에 부정적인 영향을 미칠 수 있습니다.5. 수명: 원목 사이딩은 다른 재료에 비해 수명이 짧을 수 있습니다. 특히 철근 콘크리트나 플라스틱과 같은 재료에 비해 수명이 짧을 수 있습니다.6. 재료 선택: 원목 사이딩은 다양한 종류가 있지만, 각 종류마다 

In [ ]:
generate_response(test['질문'][0],1000)

'방청 페인트의 종류에는 철분이 함유된 화합물로부터 철과 철 합금을 보호하는 방청 페인트, 철분 없이 철과 철 합금을 보호하는 방청 페인트, 철분 없이 철과 철 합금을 보호하는 방청 페인트 등 다양한 종류가 있습니다. 원목사이딩을 사용하는 단점은 비, 바람, 태양에 노출되어 손상을 입을 수 있다는 점입니다. 또한 보수 작업이 필요하고 일부 종류는 살모시나 곰팡이에 취약할 수 있습니다.'

# 추론 및 답변 저장

In [ ]:
def generate_response(user_request, max_len=1000):
    conversation = [ {'role': 'user', 'content': user_request} ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             use_cache=True,
                             max_length=max_len,
                            num_return_sequences=1,
                            do_sample=True,
                            temperature=0.87,
                            top_k=27,
                            top_p=0.7
                             )
    output_text = tokenizer.decode(outputs[0])
    output_text = output_text.split("### Assistant:")[1].strip()
    output_text = output_text.replace('\n','').replace('</s>','')

    return output_text

In [ ]:
generate_response(test['질문'][0],1000)

'방청 페인트는 방청 성분이 포함된 특수 페인트의 일종으로, 부식을 방지하고 금속 표면을 보호하는 데 사용됩니다. 방청 페인트의 일반적인 종류에는 다음이 있습니다:1. 방청 에폭시 페인트: 방청 성분과 에폭시 수지가 함유된 페인트로, 우수한 부식 방지 및 보호 특성을 제공합니다.2. 방청 아크릴 페인트: 방청 성분과 아크릴 수지가 함유된 페인트로, 내부 및 외부 금속 표면에 적합합니다.3. 방청 우레탄 페인트: 방청 성분과 우레탄 수지가 함유된 페인트로, 내부 및 외부 금속 표면에 적합합니다.4. 방청 알킬 페인트: 방청 성분과 알킬 수지가 함유된 페인트로, 특히 알루미늄 및 구리 표면에 적합합니다.원목 사이딩은 자연재로 만들어져 고급스러운 외관을 제공하는 것으로 알려져 있지만, 일부 단점이 있습니다:1. 유지 관리: 원목 사이딩은 보호 코팅이 필요하며 시간이 지남에 따라 갈라지고 변색될 수 있습니다. 따라서 시간이 지남에 따라 유지 관리가 필요합니다.2. 손상 취약성: 원목 사이딩은 손상에 취약하며 부식, 착상 및 곤충 침입에 취약합니다.3. 비용: 원목 사이딩은 다른 재료에 비해 비용이 많이 들 수 있으며, 특히 고급 재료를 사용하는 경우 더욱 비용이 많이 들 수 있습니다.4. 재활용성: 원목 사이딩은 다른 재료에 비해 재활용 가능성이 낮을 수 있습니다.요점은 원목 사이딩의 단점을 극복하기 위해 방청 페인트를 사용하면 방청 성분이 금속 표면을 보호하고 부식을 방지하는 데 도움이 될 수 있다는 점입니다.'

In [ ]:
generate_response(test['질문'][0],400)

'방청 페인트는 다양한 종류가 있으며, 그 중 일부는 다음과 같습니다:1. 방청 방지 페인트: 철제 구조물과 같은 금속 표면을 부식과 찌그러짐으로부터 보호하기 위해 설계된 페인트입니다.2. 방청 방지 코팅: 철, 알루미늄 등 금속 표면을 보호하는 데 사용되는 코팅입니다.3. 방청 코팅: 철과 같은 금속 표면을 보호하기 위해 사용되는 코팅입니다.4. 방청 방지 시스템: 금속 표면을 보호하는 데 사용되는 방청 코팅과 함께 사용되는 시스템입니다.원목 사이딩을 사용하면 다음과 같은 단점이 있을 수 있습니다:1. 유지보수: 원목 사이딩은'

In [ ]:
generate_response(test['질문'][0],800)

'방청 페인트는 방청 페인트, 방청 코팅, 방청 보호제 등 다양한 종류가 있습니다. 각 종류의 방청 페인트는 다양한 환경에서 방청 성능을 유지하는 데 도움이 되는 특수 화학 성분을 포함하고 있습니다. 예를 들어, 방청 페인트는 철 기반 표면에 적용되는 반면, 방청 코팅은 알루미늄 및 다른 금속 기반 표면에 적용됩니다.원목 사이딩을 사용하는 단점은 다음과 같습니다:1. 유지 관리: 원목 사이딩은 깨끗하고 보호되도록 규칙적으로 깨끗이 하고 보호제를 도포해야 합니다.2. 손상 및 손상: 원목 사이딩은 바람, 비, 태양에 노출되면 휨, 휨, 골 형성 등 손상을 쉽게 입을 수 있습니다.3. 재료 비용: 원목 사이딩은 다른 사이딩 재료보다 비용이 많이 들 수 있습니다.4. 내화성: 원목 사이딩은 다른 재료에 비해 내화성이 떨어질 수 있습니다.5. 수명: 원목 사이딩은 유지 관리가 부족하거나 외부 환경에 노출되면 수명이 짧을 수 있습니다.'

In [ ]:
train_ft = pd.read_csv(f'{data_path}train_80000_rule_based.csv')
train_ft

train_ft['answer'].str.len().max()

1851

In [ ]:
reset_seeds(SEED)

test_answers= []
for i in tqdm(range(len(test))):
    answer = generate_response(test['질문'][i],1000)
    print(answer)
    test_answers.append(answer)

test_answers

  1%|          | 1/130 [02:16<4:53:06, 136.33s/it]

방청 페인트는 기름, 식품, 화학물질 등 다양한 유형의 염증 유발 물질에 저항하도록 설계된 페인트의 일종입니다. 방청 페인트는 주로 산화 방지제, 안료, 바인더 및 용매로 구성됩니다. 방청 페인트의 일반적인 유형은 다음과 같습니다:1. 방청 에폭시 페인트: 산화 방지제와 안료를 포함하여 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.2. 방청 알킬 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.3. 방청 아크릴 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.4. 방청 셀룰로오스 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.5. 방청 우레탄 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.원목사이딩은 숲에서 직접 얻은 원목을 사용하여 외벽을 덮는 것을 말합니다. 원목사이딩을 사용하는 것에는 몇 가지 단점이 있습니다:1. 유지보수: 원목사이딩은 빛, 습기 및 기타 외부 요인으로 인해 손상될 수 있습니다. 따라서 원목사이딩은 일반적으로 빛과 습기에 저항하는 페인트나 스틸로 코팅해야 합니다.2. 재료 비용: 원목사이딩은 일반적으로 다른 재료보다 비싸다. 특히 고급 원목 사이딩은 더 비싸다.3. 환경적 영향: 원목사이딩을 위해 나무를 베어 내면 환경에 부정적인 영향을 미칠 수 있습니다. 특히 깊은 숲이나 보호 수역에서 나무를 베어 내면 생태계에 영향을 미


  2%|▏         | 2/130 [04:35<4:54:20, 137.97s/it]

도배지에 녹은 자국이 발생하는 주된 원인은 물, 습기 또는 수분에 노출되는 것입니다. 도배지는 일반적으로 물과 습기에 취약하므로 이러한 환경에 노출되면 부분적으로 녹아 녹은 자국이 생길 수 있습니다.이러한 문제를 해결하려면 녹은 자국이 발생한 부분을 건조하거나 제거하는 것부터 시작할 수 있습니다. 건조 과정에는 도배지를 흡수하는 건조 천을 사용하거나 냉방기 또는 선풍기를 사용하여 공기를 조정하는 것이 포함될 수 있습니다. 녹은 자국이 심하거나 제거하기 어려운 경우 도배지 전체를 제거하고 새로운 도배지를 설치하는 것이 필요할 수 있습니다.이러한 문제를 예방하려면 도배지가 물, 습기 또는 수분에 노출되지 않도록 하는 것이 중요합니다. 샤워 또는 욕조 주변에 도배지를 설치하는 경우 욕실용 도배지를 사용하고 덮개나 실링을 사용하여 샤워실 벽과 욕조 사이의 접합부를 밀봉하는 것이 좋습니다. 또한 창문과 문 주변에 도배지를 설치할 때는 덮개를 사용하여 물이 벽으로 유입되는 것을 방지하는 것이 좋습니다.또한 도배지 주변에 물이 녹아 들어가는 것을 방지하기 위해 덮개를 사용하는 것도 중요합니다. 이렇게 하면 도배지가 물과 습기에 노출되는 것을 방지하고 녹은 자국의 위험을 줄일 수 있습니다.마지막으로 도배지가 물과 습기에 노출되는 것을 방지하기 위해 건조 환경을 유지하는 것이 중요합니다. 이를 위해 환기 시스템을 설치하고 공기 중 습도를 감시하는 것이 좋습니다. 또한 물과 습기가 벽으로 유입되는 것을 방지하기 위해 배수 시스템을 유지하고 수리하는 것이 중요합니다.요약하면, 도배지에 녹은 자국이 발생하는 주된 원인은 물, 습기 또는 수분에 노출되는 것입니다. 이러한 문제를 해결하려면 녹은 자국을 건조하거나 제거하고, 도배지가 물, 습기 또는 수분에 노출되지 않도록 하고,


  2%|▏         | 3/130 [05:44<3:45:33, 106.56s/it]

...

하도재가 없이 페인트를 바로 바르면 여러 가지 문제가 발생할 수 있습니다. 하도재가 없으면 다음과 같은 문제가 발생할 수 있습니다:1. 부식: 하도재가 없으면 표면이 부식되어 손상될 수 있습니다. 하도재는 표면을 보호하여 부식으로부터 보호합니다.2. 접착력 문제: 하도재가 없으면 페인트가 표면에 접착되지 않아 갈라지거나 벗겨질 수 있습니다. 하도재는 페인트가 표면에 잘 부착되도록 도와줍니다.3. 흡수 문제: 하도재가 없으면 표면이 페인트를 흡수하여 페인트가 고르게 분포되지 않을 수 있습니다. 하도재는 표면에 페인트가 고르게 분포되도록 도와줍니다.4. 균일한 결과물이 아닙니다: 하도재가 없으면 표면이 거칠어 페인트가 균일하게 분포되지 않을 수 있습니다. 하도재는 표면을 평평하게 만들어 균일한 결과물을 얻을 수 있도록 도와줍니다.5. 지속성 문제: 하도재가 없으면 페인트가 더 빨리 마모되고 흐트러질 수 있습니다. 하도재는 페인트가 오랜 기간 동안 더 견고하게 유지되도록 도와줍니다.따라서 하도재가 없이 페인트를 바로 바르면 결과물이 균일하지 않고 견고하지 않을 수 있으므로 하도재를 사용하는 것이 중요합니다.


 28%|██▊       | 36/130 [1:01:45<2:08:35, 82.08s/it]

바닥재는 일반적으로 절단 후 잔여물로 발생합니다. 이 잔여물은 잔여물 봉지에 담아 쓰레기통에 버리거나 지역 폐기물 처리 기관에 반납할 수 있습니다.장판은 일반적으로 절단 후 잔여물로 발생합니다. 이 잔여물은 잔여물 봉지에 담아 쓰레기통에 버리거나 지역 폐기물 처리 기관에 반납할 수 있습니다.또한 잔여물을 재활용할 수 있는 경우도 있습니다. 예를 들어 잔여물을 다시 절단하여 새로운 장판을 만들거나 다른 목적으로 사용할 수 있습니다.


 28%|██▊       | 37/130 [1:04:04<2:33:25, 98.98s/it]

도배지에 생긴 반점을 제거하는 데 도움이 되는 몇 가지 방법이 있습니다. 그 중 일부는 다음과 같습니다:1. 사포 또는 세제를 사용하여 손으로 문지르기: 손으로 도배지에 생긴 반점을 사포 또는 세제로 문지르면 일부 반점을 제거할 수 있습니다. 하지만 이 방법은 많은 노력과 시간이 소요될 수 있습니다.2. 휘발성 화학 물질을 사용하여 문지르기: 일부 화학 물질은 도배지에 생긴 반점을 제거하는 데 도움이 될 수 있습니다. 그러나 이 방법은 화학 물질이 반점을 제거하는 동안 도배지에 손상을 줄 수 있으므로 주의해야 합니다.3. 도배지를 깨끗이 닦기: 도배지를 깨끗이 닦으면 반점이 제거될 수 있습니다. 그러나 이 방법은 많은 노력과 시간이 소요될 수 있으며 도배지에 손상을 줄 수 있습니다.4. 화학 물질을 사용하여 문지르기: 일부 화학 물질은 도배지에 생긴 반점을 제거하는 데 도움이 될 수 있습니다. 그러나 이 방법은 화학 물질이 반점을 제거하는 동안 도배지에 손상을 줄 수 있으므로 주의해야 합니다.5. 도배지를 깨끗이 닦기: 도배지를 깨끗이 닦으면 반점이 제거될 수 있습니다. 그러나 이 방법은 많은 노력과 시간이 소요될 수 있으며 도배지에 손상을 줄 수 있습니다.6. 화학 물질을 사용하여 문지르기: 일부 화학 물질은 도배지에 생긴 반점을 제거하는 데 도움이 될 수 있습니다. 그러나 이 방법은 화학 물질이 반점을 제거하는 동안 도배지에 손상을 줄 수 있으므로 주의해야 합니다.7. 도배지를 깨끗이 닦기: 도배지를 깨끗이 닦으면 반점이 제거될 수 있습니다. 그러나 이 방법은 많은 노력과 시간이 소요될 수 있으며 도배지에 손상을 줄 수 있습니다.8. 화학 물질을 사용하여 문지르기: 일부 화학 물질은 도배지에


In [ ]:
import pickle
# 리스트를 피클 파일로 저장
with open('test_answers_tuned_max1000_megast_0302.pkl', 'wb') as f:
    pickle.dump(test_answers, f)

In [ ]:
for a, b in zip(test['질문'].tolist(), test_answers) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(' ')

In [ ]:
# Embedding Vector 추출
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1') # 평가를 위한 Embedding Vector 추출에 활용하는 모델

pred_vector = vector_model.encode(test_answers)
pred_vector.shape

In [ ]:
submission = pd.read_csv(f'{data_path}sample_submission.csv')
submission

In [ ]:
submission.iloc[:,1:] = pred_vector
submission

In [ ]:
submission.to_csv(f'{data_path}submission_mega0302_epoch1_max1000_0303.csv', index=False)
submission.to_csv(f'submission_mega0302_epoch1_max1000_0303.csv', index=False)



# 추론 및 답변 저장 2

In [ ]:
def chatbot(model, tokenizer, device):
    preds = []
    for i in tqdm(range(test.shape[0])):
        test_question = test['질문'][i]
        test_question = "<s>[INST] " + test_question + " [/INST]"
        input_ids = tokenizer.encode(test_question, return_tensors='pt').to(device)
        result_ids = model.generate(input_ids=input_ids.to(device),
                                    max_length=300,
                                    repetition_penalty=1.3,
                                    num_beams=1,
                                    num_return_sequences=1,
                                    no_repeat_ngram_size=4,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.1,
                                    top_k=90,
                                    top_p=0.95)
        q_len = len(test_question)
        result = tokenizer.decode(result_ids[0])
        preds.append(result[q_len+4])
    return preds

In [ ]:
preds = chatbot(model, tokenizer, device)
preds

In [ ]:
import pandas as pd
test = pd.read_csv(f'{data_path}test.csv')
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


# 추론 및 답변 저장 3

### tuned 모델 : max_len =2048 추론

In [ ]:
import random
import numpy as np
from tqdm import tqdm

reset_seeds(SEED)

test_answers= []
for i in tqdm(range(len(test))):
    answer = generate_response(test['질문'][i],2048)
    print(answer)
    test_answers.append(answer)

test_answers

  1%|          | 1/130 [02:39<5:43:05, 159.58s/it]

방청 페인트는 기름, 식품, 화학물질 등 다양한 유형의 염증 유발 물질에 저항하도록 설계된 페인트의 일종입니다. 방청 페인트는 주로 산화 방지제, 안료, 바인더 및 용매로 구성됩니다. 방청 페인트의 일반적인 유형은 다음과 같습니다:1. 방청 에폭시 페인트: 산화 방지제와 안료를 포함하여 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.2. 방청 알킬 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.3. 방청 아크릴 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.4. 방청 셀룰로오스 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.5. 방청 우레탄 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.원목사이딩은 숲에서 직접 얻은 원목을 사용하여 외벽을 덮는 것을 말합니다. 원목사이딩을 사용하는 것에는 몇 가지 단점이 있습니다:1. 유지보수: 원목사이딩은 빛, 습기 및 기타 외부 요인으로 인해 손상될 수 있습니다. 따라서 원목사이딩은 일반적으로 빛과 습기에 저항하는 페인트나 스틸로 코팅해야 합니다.2. 재료 비용: 원목사이딩은 일반적으로 다른 재료보다 비싸다. 특히 고급 원목 사이딩은 더 비싸다.3. 환경적 영향: 원목사이딩을 위해 나무를 베어 내면 환경에 부정적인 영향을 미칠 수 있습니다. 특히 깊은 숲이나 보호 수역에서 나무를 베어 내면 생태계에 영향을 미칠 수 있습니다.4. 제한된 색상 및 디자인 옵션: 원목사이딩은 자연 상태의 나무 색상만 사용할 수 있기 때문에 색상 및 디자인 옵션이 제한됩니다.전반적으로 원목사이딩은 높은 유지보수 비용, 비싼 재료 비용, 환경 영향, 제한된 색상 및 디자인 옵션 등 몇 가지 단점이 있습니다.


  2%|▏         | 2/130 [05:08<5:26:41, 153.13s/it]

도배지에 녹은 자국이 발생하는 주요 원인은 도배지 표면에 물이 침투하여 섬유가 부드러워지고 쉽게 찢어지기 때문입니다. 이러한 문제는 물 손상, 먼지 및 먼지 침착, 자외선 노출 등 여러 가지 요인으로 인해 발생할 수 있습니다.녹은 자국을 해결하려면 다음 단계를 따르세요:1. 물 손상: 도배지가 젖었는지 확인하세요. 젖은 도배지는 쉽게 찢어지고 녹은 자국이 생길 수 있습니다. 물 손상이 발생했다면 도배지를 건조시키고 다시 붙여야 합니다.2. 먼지 및 먼지 침착: 도배지 표면에 먼지나 먼지가 쌓이면 섬유가 부드러워져 찢어질 수 있습니다. 먼지와 먼지를 닦아내면 녹은 자국을 방지할 수 있습니다.3. 자외선 노출: 도배지가 많은 양의 자외선에 노출되면 섬유가 손상되고 녹은 자국이 생길 수 있습니다. 도배지를 직사광선에 노출되지 않는 곳에 배치하면 녹은 자국을 방지할 수 있습니다.4. 녹은 자국 수리: 도배지에 녹은 자국이 생긴 경우 수리하는 것이 가장 좋습니다. 먼저 녹은 자국 주변의 도배지를 제거한 다음 수리 패치를 붙입니다. 수리 패치는 도배지와 일치하는 색상과 질감을 가져야 합니다. 수리 패치를 도배지 손상 부위에 부드럽게 눌러서 부착시킵니다.5. 도배지 보호: 녹은 자국이 생기지 않도록 도배지를 보호하는 것이 가장 좋습니다. 도배지를 건조하고 청결하게 유지하고, 직사광선에서 멀리 떨어져 있도록 하고, 먼지나 먼지가 쌓이지 않도록 하세요.요약하면, 도배지에 녹은 자국이 발생하는 주요 원인은 물 손상, 먼지 및 먼지 침착, 자외선 노출입니다. 물 손상이나 먼지 및 먼지 침착을 방지하기 위해 도배지를 건조하고 청결하게 유지하고, 도배지를 직사광선에서 멀리 떨어져 있도록 하세요. 녹은 자국이 생긴 경우 수리하고 보호하세요.


  2%|▏         | 3/130 [06:13<3:59:38, 113.22s/it]

...

네, 물론이죠! 석고수정은 옥수수가 수분이 많아 발생하는 현상입니다. 옥수수가 많은 수분을 흡수하면 수정처럼 보이는 석고수정이 생길 수 있습니다. 이는 옥수수가 과도한 수분을 흡수하면 발생하는 일반적인 현상입니다.피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는 피스 하자는

 17%|█▋        | 22/130 [57:50<4:37:53, 154.38s/it]

공동주택의 실내 공기질을 측정할 때 어떤 층에서 샘플을 채취해야 하는지는 측정하려는 특정 대상에 따라 달라집니다. 공기 중의 대기 오염 물질이나 화학 물질을 측정하려면 층의 가장 낮은 부분에서 샘플을 채취하는 것이 일반적입니다. 이는 공기가 앙징 현상을 일으키고 가장 낮은 부분으로 가라앉기 때문입니다. 이 부분에서 샘플을 채취하면 공기 중의 오염 물질 농도가 가장 높아질 것입니다.하지만 특정 공동주택 층의 특정 공기 중 오염 물질이나 화학 물질에 대한 정보를 원한다면 해당 층의 공기 중 오염 물질 농도를 정확하게 측정하기 위해 해당 층에서 샘플을 채취하는 것이 좋습니다.


 17%|█▋        | 22/130 [1:00:10<4:55:26, 164.14s/it]


KeyboardInterrupt: 

In [ ]:
import pickle
# 리스트를 피클 파일로 저장
with open('test_answers_tuned_max2048_megast_0302.pkl', 'wb') as f:
    pickle.dump(test_answers, f)

In [ ]:
for a, b in zip(test['질문'].tolist(), test_answers) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청 페인트는 기름, 식품, 화학물질 등 다양한 유형의 염증 유발 물질에 저항하도록 설계된 페인트의 일종입니다. 방청 페인트는 주로 산화 방지제, 안료, 바인더 및 용매로 구성됩니다. 방청 페인트의 일반적인 유형은 다음과 같습니다:1. 방청 에폭시 페인트: 산화 방지제와 안료를 포함하여 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.2. 방청 알킬 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.3. 방청 아크릴 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.4. 방청 셀룰로오스 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.5. 방청 우레탄 페인트: 산화 방지제, 안료 및 바인더로 구성되며 고무, 섬유 및 섬유 기반 물질에 적용하여 방청 및 방청 기능을 제공합니다.원목사이딩은 숲에서 직접 얻은 원목을 사용하여 외벽을 덮는 것을 말합니다. 원목사이딩을 사용하는 것에는 몇 가지 단점이 있습니다:1. 유지보수: 원목사이딩은 빛, 습기 및 기타 외부 요인으로 인해 손상될 수 있습니다. 따라서 원목사이딩은 일반적으로 빛과 습기에 저항하는 페인트나 스틸로 코팅해야 합니다.2. 재료 비용: 원목사이딩은 일반적으로 다른 재료보다 비싸다. 특히 고급 원목 사이딩은 더 비싸다.3. 환경적 영향: 원목사이딩을 위해 나무를 베어 내면 환경에 부정적인 영향을 미칠 수 있습니다. 특히 깊은 숲이나 보호 수역에서 나무를 베어 내면 생태계에 영향을 미칠 수 있습니다.4. 제한된 색상 및 디자인 옵션: 원목사이딩은 자연 상태의 나무 색상만 사용할 수 있기 때문에 색상 및 디자인 옵션이 제한됩니다.전

In [ ]:
# Embedding Vector 추출
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1') # 평가를 위한 Embedding Vector 추출에 활용하는 모델

pred_vector = vector_model.encode(test_answers)
pred_vector.shape

In [ ]:
submission = pd.read_csv(f'{data_path}sample_submission.csv')
submission

In [ ]:
submission.iloc[:,1:] = pred_vector
submission

In [ ]:
submission.to_csv(f'{data_path}submission_mega0302_epoch1_max2048_0303.csv', index=False)
submission.to_csv(f'submission_mega0302_epoch1_max2048_0303.csv', index=False)

